# Advanced Housing Prices: Feature Engineering 
The Ames Housing dataset was compiled by Dean De Cock for use in data science education. It's an incredible alternative for data scientists looking for a modernized and expanded version of the often cited Boston Housing dataset. 

The broad aims of this Machine Learning Project are to understand the dataset and to build a ML pripeline that can predict SalePrice of Houses using various techniques.

The notebooks for different pipes can be found in the this github [repo.](https://github.com/hantablack9/House-Price-Prediction-kaggle)

The dataset and documentation can be found in these links:
[1. Kaggle](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques)
[2. Feature documentation by Dean de Cock](https://jse.amstat.org/v19n3/decock/DataDocumentation.txt)


## Some Insights from the ['Ames, Iowa: Alternative to the Boston Housing Data as an End of Semester Regression Project'](https://jse.amstat.org/v19n3/decock.pdf) by Dean De Cock, Truman State University


1.  The dataset contains 2930 observations and a set of 80 explanatory variables (23 nominal, 23 ordinal, 14 discrete, and 20 continuous) involved in assessing home values.

2. **20 continuous variables** relate to various area dimensions for each observation.

3. The **14 discrete variables** typically quantify the number of items occurring within the house. Most are specifically focused on the number of kitchens, bedrooms, and bathrooms, etc.

4. There are a large number of **categorical variables (23 nominal, 23 ordinal)** associated with this data set. They range from 2 to 28 classes with the smallest being STREET (gravel or paved) and the largest being NEIGHBORHOOD (areas within the Ames city limits). The nominal variables typically identify various types of dwellings, garages, materials, and environmental conditions while the ordinal variables typically rate various items within the property.

4. **Outliers**, bad datapoints: 5 observations to be removed from the data (**a plot of SALE PRICE versus GR LIV AREA** will quickly indicate these points). Three of them are true outliers (Partial Sales that likely don’t represent actual market values) and two of them are simply unusual sales (very large houses priced relatively appropriately). Any houses with more than 4000 square feet from the data set (which eliminates these five unusual observations).

**We will perform feature engineering in this notebook.**

We will also look into and perform the folowing for:
1. Missing value fixes : Temporal, Categorical, Numeric features
2. Feature scaling

Preceding steps of this data pipeline are in different notebooks and can be accessed from this github [repo.](https://github.com/hantablack9/House-Price-Prediction-kaggle/tree/main)


In [5]:
import pandas as pd
import numpy as np

import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt

from scipy.stats import skew

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.metrics import mean_squared_error, make_scorer

from IPython.display import display

%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.pandas.set_option('display.float_format', lambda x: '%.3f' % x)

In [6]:
df_tr = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv", index_col = 'Id')
df_t = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv", index_col = 'Id')

In [7]:
df_tr.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.000,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.000,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.000,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.000,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.000,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.000,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.000,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.000,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.000,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.000,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.000,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.000,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.000,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.000,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.000,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
df_t.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


## 0. Clean-up

Lets perform a little housekeeping to make the features manageable.

In [10]:
df_tr['MSSubClass'].unique(), df_t['MSSubClass'].unique()

(array([ 60,  20,  70,  50, 190,  45,  90, 120,  30,  85,  80, 160,  75,
        180,  40]),
 array([ 20,  60, 120, 160,  80,  30,  50,  90,  85, 190,  45,  70,  75,
        180,  40, 150]))

## 1. Missing Values

In [5]:
## Missing Categorical Values in training data
cat_na_features = [features for features in df_tr.columns if df_tr[features].isnull().sum()>1 and df_tr[features].dtypes=='O']

print('======= Train data: Categorical Features with Missing Values =======')
for feature in cat_na_features:
    print(feature,'\t\t', np.round(df_tr[feature].isnull().mean(),3), '%')

======= Train data: Categorical Features with Missing Values =======
Alley 		 0.938 %
MasVnrType 		 0.005 %
BsmtQual 		 0.025 %
BsmtCond 		 0.025 %
BsmtExposure 		 0.026 %
BsmtFinType1 		 0.025 %
BsmtFinType2 		 0.026 %
FireplaceQu 		 0.473 %
GarageType 		 0.055 %
GarageFinish 		 0.055 %
GarageQual 		 0.055 %
GarageCond 		 0.055 %
PoolQC 		 0.995 %
Fence 		 0.808 %
MiscFeature 		 0.963 %


In [6]:
## Missing Categorical Values in test data
test_cat_na_features = [features for features in df_t.columns if df_t[features].isnull().sum()>1 and df_t[features].dtypes=='O']

print('======= Test data: Categorical Features with Missing Values =======')
for feature in test_cat_na_features:
    print(feature,'\t\t', np.round(df_t[feature].isnull().mean(),3), '%')

======= Test data: Categorical Features with Missing Values =======
MSZoning 		 0.003 %
Alley 		 0.927 %
Utilities 		 0.001 %
MasVnrType 		 0.011 %
BsmtQual 		 0.03 %
BsmtCond 		 0.031 %
BsmtExposure 		 0.03 %
BsmtFinType1 		 0.029 %
BsmtFinType2 		 0.029 %
Functional 		 0.001 %
FireplaceQu 		 0.5 %
GarageType 		 0.052 %
GarageFinish 		 0.053 %
GarageQual 		 0.053 %
GarageCond 		 0.053 %
PoolQC 		 0.998 %
Fence 		 0.801 %
MiscFeature 		 0.965 %


In [7]:
## Train data : Custom replace funcion
def replace_cat_missing_vals(dataset, na_features):
    data = dataset.copy()
    data[na_features] = data[na_features].fillna('Missing')
    
    return data
df_tr = replace_cat_missing_vals(df_tr, cat_na_features)

df_tr[cat_na_features].isnull().sum()

Alley           0
MasVnrType      0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinType2    0
FireplaceQu     0
GarageType      0
GarageFinish    0
GarageQual      0
GarageCond      0
PoolQC          0
Fence           0
MiscFeature     0
dtype: int64

In [8]:
## Train data: Missing Numeric Variables

train_num_na_features = [features for features in df_tr.columns if df_tr[features].isnull().sum()>1 and df_tr[features].dtypes!='O']

print('======= Numeric Features with Missing Values =======')
for feature in train_num_na_features:
    print(feature,'\t\t', np.round(df_tr[feature].isnull().mean(),3), '%')## Replace 

======= Numeric Features with Missing Values =======
LotFrontage 		 0.177 %
MasVnrArea 		 0.005 %
GarageYrBlt 		 0.055 %


In [9]:
## Train data: Custom replace for missing values 
for feature in train_num_na_features:
    #using medians because of presence of outliers
    median_value = df_tr[feature].median()
    
    #create new features to capture nan values
    df_tr[feature+' nan'] = np.where(df_tr[feature].isnull(),1,0)
    df_tr[feature].fillna(median_value, inplace=True)

df_tr[train_num_na_features].isnull().sum()

LotFrontage    0
MasVnrArea     0
GarageYrBlt    0
dtype: int64

In [10]:
year_features=['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']

for feature in year_features:
    df_tr[feature]=df_tr['YrSold']-df_tr[feature]

In [11]:
year_features=['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']

for feature in year_features:
    df_t[feature]=df_t['YrSold']-df_t[feature]

In [12]:
df_tr[year_features]

,YearBuilt,YearRemodAdd,GarageYrBlt
Id,,,
1,5,5,5.0
2,31,31,31.0
3,7,6,7.0
4,91,36,8.0
5,8,8,8.0
...,...,...,...
1456,8,7,8.0
1457,32,22,32.0
1458,69,4,69.0


In [13]:
df_t[year_features]

,YearBuilt,YearRemodAdd,GarageYrBlt
Id,,,
1461,49,49,49.0
1462,52,52,52.0
1463,13,12,13.0
1464,12,12,12.0
1465,18,18,18.0
...,...,...,...
2915,36,36,NaN
2916,36,36,36.0
2917,46,10,46.0


## 2. Feature Scaling